In [4]:
import sys, subprocess, importlib
def pip_try(*pkgs):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])
        print("✅", " ".join(pkgs))
    except Exception as e:
        print("⚠️  skip", " ".join(pkgs), "->", e)

pip_try("pandas", "numpy", "scikit-learn", "matplotlib", "joblib", "scipy")
import sklearn, numpy, pandas, scipy
print("Python:", sys.executable)
print("sklearn:", sklearn.__version__)

✅ pandas numpy scikit-learn matplotlib joblib scipy
Python: c:\Users\hamas\AppData\Local\Programs\Python\Python313\python.exe
sklearn: 1.7.1


In [5]:
import os, re, json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Dict, Any, List

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import scipy.sparse as sp
import joblib

try:
    from IPython.display import display
except Exception:
    pass

In [ ]:
DATA_PATH = "./dataset.csv"

TARGET_COL = "Role"

if DATA_PATH is None:
    csvs = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if len(csvs) == 1:
        DATA_PATH = csvs[0]
        print(f"Auto-detected dataset: {DATA_PATH}")
    elif len(csvs) > 1:
        picks = [f for f in csvs if re.search(r'career|map|skill|job|role', f, re.I)]
        if picks:
            DATA_PATH = picks[0]
            print(f"Auto-detected dataset (heuristic): {DATA_PATH}")
        else:
            print("Multiple CSV ditemukan. Set DATA_PATH manual.")
    else:
        print("Tidak ada file .csv. Set DATA_PATH manual.")

if DATA_PATH is None:
    raise FileNotFoundError("Set DATA_PATH ke file dataset kamu (CSV/XLSX).")

if DATA_PATH.lower().endswith('.csv'):
    df = pd.read_csv(DATA_PATH)
elif DATA_PATH.lower().endswith(('.xlsx', '.xls')):
    df = pd.read_excel(DATA_PATH)

print("Shape:", df.shape)
try: display(df.head())
except: print(df.head())

Shape: (9179, 28)


,Database Fundamentals,Computer Architecture,Distributed Computing Systems,Cyber Security,Networking,Software Development,Programming Skills,Project Management,Computer Forensics Fundamentals,Technical Communication,...,Conscientousness,Extraversion,Agreeableness,Emotional_Range,Conversation,Openness to Change,Hedonism,Self-enhancement,Self-transcendence,Role
0,6,4,4,4,4,4,4,4,4,4,...,0.480564,0.470877,0.039611,0.742567,0.086927,0.339776,0.091158,0.230518,0.208746,Database Administrator
1,6,5,4,4,4,4,4,4,4,4,...,0.452903,0.377581,0.090148,0.672546,0.071425,0.353678,0.104495,0.247432,0.196424,Database Administrator
2,6,1,4,4,4,4,4,4,4,4,...,0.461668,0.347739,0.138575,0.630705,0.095261,0.375125,0.135462,0.287253,0.268024,Database Administrator
3,6,0,4,4,4,4,4,4,4,4,...,0.521783,0.403756,0.114925,0.695280,0.122656,0.338899,0.161705,0.306562,0.258119,Database Administrator
4,6,3,4,4,4,4,4,4,4,4,...,0.319539,0.240337,0.050988,0.619390,0.065564,0.311896,0.147700,0.302978,0.207287,Database Administrator


In [ ]:
if TARGET_COL is None:
    candidates = [c for c in df.columns if re.search(r'(job|role|position|pekerjaan|label|target)', c, re.I)]
    TARGET_COL = candidates[0] if candidates else df.columns[-1]
print("Target column:", TARGET_COL)

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]
print("Numerik:", num_cols[:10], "..." if len(num_cols)>10 else "")
print("Kategorikal:", cat_cols[:10], "..." if len(cat_cols)>10 else "")

numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocess = ColumnTransformer([
    ("num", numeric_transformer, num_cols),
    ("cat", categorical_transformer, cat_cols),
])
feature_columns = list(X.columns)

Target column: Role
Numerik: ['Database Fundamentals', 'Computer Architecture', 'Distributed Computing Systems', 'Cyber Security', 'Networking', 'Software Development', 'Programming Skills', 'Project Management', 'Computer Forensics Fundamentals', 'Technical Communication'] ...
Kategorikal: [] 


In [8]:
def analyze_and_bin_numeric_columns(df, numeric_cols):
    binning_info = {}
    
    for col in numeric_cols:
        col_data = df[col].dropna()
        if len(col_data) == 0:
            continue
            
        min_val = col_data.min()
        max_val = col_data.max()
        
        print(f"\n{col}:")
        print(f"  Range: [{min_val:.2f}, {max_val:.2f}]")
        print(f"  Mean: {col_data.mean():.2f}, Median: {col_data.median():.2f}")
        
        if min_val >= 1 and max_val <= 5 and col_data.nunique() <= 5:
            print(f"  ✓ Sudah dalam range 1-5, skip binning")
            binning_info[col] = None
            continue
        
        bins = [-np.inf]
        for q in [0.2, 0.4, 0.6, 0.8]:
            bins.append(col_data.quantile(q))
        bins.append(np.inf)
        
        bins = sorted(list(set(bins)))
        
        labels = list(range(1, len(bins)))
        binning_info[col] = {'bins': bins, 'labels': labels}
        
        print(f"  → Bins: {[f'{b:.2f}' for b in bins]}")
        print(f"  → Labels: {labels}")
    
    return binning_info

print("="*60)
print("ANALISIS KOLOM NUMERIK UNTUK BINNING")
print("="*60)
binning_info = analyze_and_bin_numeric_columns(X, num_cols)

ANALISIS KOLOM NUMERIK UNTUK BINNING

Database Fundamentals:
  Range: [0.00, 6.00]
  Mean: 2.71, Median: 3.00
  → Bins: ['-inf', '1.00', '2.00', '3.00', '5.00', 'inf']
  → Labels: [1, 2, 3, 4, 5]

Computer Architecture:
  Range: [0.00, 6.00]
  Mean: 2.71, Median: 3.00
  → Bins: ['-inf', '1.00', '2.00', '3.00', '5.00', 'inf']
  → Labels: [1, 2, 3, 4, 5]

Distributed Computing Systems:
  Range: [0.00, 6.00]
  Mean: 2.71, Median: 3.00
  → Bins: ['-inf', '1.00', '2.00', '3.00', '5.00', 'inf']
  → Labels: [1, 2, 3, 4, 5]

Cyber Security:
  Range: [0.00, 6.00]
  Mean: 2.71, Median: 3.00
  → Bins: ['-inf', '1.00', '2.00', '3.00', '5.00', 'inf']
  → Labels: [1, 2, 3, 4, 5]

Networking:
  Range: [0.00, 6.00]
  Mean: 2.71, Median: 3.00
  → Bins: ['-inf', '1.00', '2.00', '3.00', '5.00', 'inf']
  → Labels: [1, 2, 3, 4, 5]

Software Development:
  Range: [0.00, 6.00]
  Mean: 2.71, Median: 3.00
  → Bins: ['-inf', '1.00', '2.00', '3.00', '5.00', 'inf']
  → Labels: [1, 2, 3, 4, 5]

Programming Skills:

In [ ]:
def apply_binning(X, binning_info):
    X_binned = X.copy()
    
    for col, info in binning_info.items():
        if info is None: 
            continue
            
        bins = info['bins']
        labels = info['labels']
        
        X_binned[col] = pd.cut(
            X[col], 
            bins=bins, 
            labels=labels, 
            include_lowest=True,
            duplicates='drop'
        )
        
        X_binned[col] = pd.to_numeric(X_binned[col], errors='coerce')
    
    return X_binned

X_binned = apply_binning(X, binning_info)

print("\n" + "="*60)
print("HASIL BINNING")
print("="*60)
for col in num_cols:
    if binning_info.get(col) is not None:
        print(f"\n{col}:")
        print(f"  Range baru: [{X_binned[col].min():.0f}, {X_binned[col].max():.0f}]")
        print(f"  Distribusi: \n{X_binned[col].value_counts().sort_index()}")

X = X_binned

print("\n✅ Binning selesai! Semua kolom numerik sekarang dalam range 1-5")


HASIL BINNING

Database Fundamentals:
  Range baru: [1, 5]
  Distribusi: 
Database Fundamentals
1    2880
2    1439
3    1440
4    2880
5     540
Name: count, dtype: int64

Computer Architecture:
  Range baru: [1, 5]
  Distribusi: 
Computer Architecture
1    2880
2    1439
3    1440
4    2880
5     540
Name: count, dtype: int64

Distributed Computing Systems:
  Range baru: [1, 5]
  Distribusi: 
Distributed Computing Systems
1    2880
2    1439
3    1440
4    2880
5     540
Name: count, dtype: int64

Cyber Security:
  Range baru: [1, 5]
  Distribusi: 
Cyber Security
1    2880
2    1439
3    1440
4    2880
5     540
Name: count, dtype: int64

Networking:
  Range baru: [1, 5]
  Distribusi: 
Networking
1    2880
2    1439
3    1440
4    2880
5     540
Name: count, dtype: int64

Software Development:
  Range baru: [1, 5]
  Distribusi: 
Software Development
1    2880
2    1439
3    1440
4    2880
5     540
Name: count, dtype: int64

Programming Skills:
  Range baru: [1, 5]
  Distribusi: 
Pr

In [ ]:
output_path = "dataset_normalized.csv"

df_normalized = X_binned.copy()
df_normalized[TARGET_COL] = y

df_normalized.to_csv(output_path, index=False)

print("\n" + "="*60)
print("SAVE NORMALIZED DATASET")
print("="*60)
print(f"✅ Dataset saved to: {output_path}")
print(f"📊 Shape: {df_normalized.shape}")
print(f"📋 Columns: {list(df_normalized.columns)}")

print("\n📌 Sample data (first 5 rows):")
print(df_normalized.head())

print("\n🔍 Verification - Range for each numeric column:")
for col in num_cols:
    if binning_info.get(col) is not None:
        print(f"  {col}: [{df_normalized[col].min():.0f}, {df_normalized[col].max():.0f}]")


SAVE NORMALIZED DATASET
✅ Dataset saved to: dataset_normalized.csv
📊 Shape: (9179, 28)
📋 Columns: ['Database Fundamentals', 'Computer Architecture', 'Distributed Computing Systems', 'Cyber Security', 'Networking', 'Software Development', 'Programming Skills', 'Project Management', 'Computer Forensics Fundamentals', 'Technical Communication', 'AI ML', 'Software Engineering', 'Business Analysis', 'Communication skills', 'Data Science', 'Troubleshooting skills', 'Graphics Designing', 'Openness', 'Conscientousness', 'Extraversion', 'Agreeableness', 'Emotional_Range', 'Conversation', 'Openness to Change', 'Hedonism', 'Self-enhancement', 'Self-transcendence', 'Role']

📌 Sample data (first 5 rows):
   Database Fundamentals  Computer Architecture  \
0                      5                      4   
1                      5                      4   
2                      5                      1   
3                      5                      1   
4                      5                   